In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cyberguard/test1.csv
/kaggle/input/cyberguard/train1.csv
/kaggle/input/final-train-cybergraud/final_train.csv


In [2]:
# CLEANING THE DATASET
# 1. removing all duplicates
df = pd.read_csv('/kaggle/input/final-train-cybergraud/final_train.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df


,Unnamed: 0,category,sub_category,crimeaditionalinfo
0,79181,Any Other Cyber Crime,Other,mera debit card mere husband ke pass tha joki ...
1,44317,Sexually Obscene material,NaN,I clicked on an escort service website and sta...
2,6843,Online Financial Fraud,UPI Related Frauds,Respected sir or madam\r\nI am A DhanushPh no ...
3,34702,Online and Social Media Related Crime,FakeImpersonating Profile,Someone is using my photo and name and created...
4,95645,Sexually Explicit Act,NaN,"Maine ek galat number pe payment kiya, jiska o..."
...,...,...,...,...
96585,6265,Sexually Obscene material,NaN,My money is dedicated please help me please re...
96586,54886,Online Financial Fraud,EWallet Related Fraud,gaw dabli khurd ward no dishanumangarh pstalwa...
96587,76820,Online Financial Fraud,Internet Banking Related Fraud,A message came from Facebook id named about a ...
96588,860,Online Financial Fraud,Fraud CallVishing,My issue is going on with samsung for repair o...


In [3]:
# df = pd.read_csv('/kaggle/input/cyberguard/train1.csv')
# df = df.groupby('category').apply(lambda x: x.drop_duplicates(subset='crimeaditionalinfo')).reset_index(drop=True)
# df
# THIS REMOVES THE DUPLICATES IN EACH CATEGORY

In [4]:
# df[(df['category'] == 'Any Other Cyber Crime') & (df['sub_category'] == 'Other')]
# # ALL OTHER CYBER CRIME ARE OTHER

In [5]:
# df = df.drop_duplicates(subset='crimeaditionalinfo')
# df

In [6]:
# test = pd.read_csv('/kaggle/input/cyberguard/test1.csv')
# test

In [7]:
df['category'].value_counts()
# # online fin fraud, RGR , 

category
Online Financial Fraud                                  57434
Online and Social Media Related Crime                   12140
Any Other Cyber Crime                                   10878
Cyber Attack/ Dependent Crimes                           3608
Sexually Explicit Act                                    3000
RapeGang Rape RGRSexually Abusive Content                2822
Sexually Obscene material                                1838
Hacking  Damage to computercomputer system etc           1710
Online Gambling  Betting                                 1000
Cyber Terrorism                                           500
Online Cyber Trafficking                                  500
Cryptocurrency Crime                                      480
Child Pornography CPChild Sexual Abuse Material CSAM      379
Ransomware                                                300
Report Unlawful Content                                     1
Name: count, dtype: int64

In [8]:
# test.value_counts('category')

In [9]:
# df['newlines'] = df['crimeaditionalinfo'].str.count('ID')
# df[df['newlines'] > 1].value_counts('category')
# ID string literally identifies 85% of SA content cases

In [10]:
# df[df['category']=='Ransomware']
# import pandas as pd
# from collections import Counter
# import matplotlib.pyplot as plt

# # Filter the DataFrame to only include rows where the category is 'Ransomware'
# ransomware_df = df[df['category'] == 'Ransomware']

# # Join all the strings in the 'category' column into a single string
# all_text = ' '.join(ransomware_df['crimeaditionalinfo'].values)

# # Split the string into a list of words
# words = all_text.split()

# # Count the occurrences of each word
# word_counts = Counter(words)

# # Get the top 10 most common words
# top_words = word_counts.most_common(100)

# # Create a DataFrame from the top words
# top_words_df = pd.DataFrame(top_words, columns=['word', 'count'])

# # Plot the top words
# plt.figure(figsize=(10, 96))
# plt.barh(top_words_df['word'], top_words_df['count'])
# plt.xlabel('Count')
# plt.ylabel('Word')
# plt.title('Top 10 most common words in Ransomware category')
# plt.gca().invert_yaxis()
# plt.show()



# for RANSOMWARE
# most interesting are files, data, ransomware, encrypted


In [11]:
# df[df['crimeaditionalinfo'].str.count('Respected Sir') > 0] #value_counts('category')
# LOOKS LIKE THERE IS MAJOR OVERLAP BETWEEN RANSOMWARE, FINANCIAL FRAUD, sOCIAL MEDIA CRIME, OTHER CYBER CRIME
# there is a need for feature enginnering to seperate online financial fraud


In [12]:
# df.iloc[25]['crimeaditionalinfo']

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
tokenizer = AutoTokenizer.from_pretrained('l3cube-pune/hing-roberta')



tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
df_binary = df.copy()
df_binary['label'] = df_binary['category'].apply(lambda x: 1 if x == "Online Financial Fraud" else 0)
df_binary['length'] = df_binary['crimeaditionalinfo'].str.len()
df_binary

,Unnamed: 0,category,sub_category,crimeaditionalinfo,label,length
0,79181,Any Other Cyber Crime,Other,mera debit card mere husband ke pass tha joki ...,0,550.0
1,44317,Sexually Obscene material,NaN,I clicked on an escort service website and sta...,0,512.0
2,6843,Online Financial Fraud,UPI Related Frauds,Respected sir or madam\r\nI am A DhanushPh no ...,1,996.0
3,34702,Online and Social Media Related Crime,FakeImpersonating Profile,Someone is using my photo and name and created...,0,893.0
4,95645,Sexually Explicit Act,NaN,"Maine ek galat number pe payment kiya, jiska o...",0,238.0
...,...,...,...,...,...,...
96585,6265,Sexually Obscene material,NaN,My money is dedicated please help me please re...,0,212.0
96586,54886,Online Financial Fraud,EWallet Related Fraud,gaw dabli khurd ward no dishanumangarh pstalwa...,1,587.0
96587,76820,Online Financial Fraud,Internet Banking Related Fraud,A message came from Facebook id named about a ...,1,1010.0
96588,860,Online Financial Fraud,Fraud CallVishing,My issue is going on with samsung for repair o...,1,617.0


In [15]:
# import matplotlib.pyplot as plt
# plt.hist(df_binary['length'])

In [16]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_binary['crimeaditionalinfo'], df_binary['label'], test_size=0.2, stratify=df_binary['label']
)

In [17]:
type(train_texts.tolist())

list

In [18]:
train_texts = [str(text) for text in train_texts]
val_texts = [str(text) for text in val_texts]

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
print('done')

done


In [19]:
 class FraudDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
from torch.utils.data import DataLoader

train_dataset = FraudDataset(train_encodings, train_labels.to_list())
val_dataset = FraudDataset(val_encodings, val_labels.to_list())


# # Create DataLoader for training dataset with shuffling enabled
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # Create DataLoader for validation dataset without shuffling (commonly not shuffled)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [20]:
binary_model = AutoModelForSequenceClassification.from_pretrained('l3cube-pune/hing-roberta', num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    save_steps=15_000,
    save_total_limit=2,
    report_to = []
)

trainer = Trainer(
    model=binary_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.379500,0.379407
2,0.345300,0.334417


TrainOutput(global_step=19318, training_loss=0.38438827091275884, metrics={'train_runtime': 9732.6455, 'train_samples_per_second': 15.879, 'train_steps_per_second': 1.985, 'total_flos': 4.066223493955584e+16, 'train_loss': 0.38438827091275884, 'epoch': 2.0})

In [21]:
print('1')

1


In [22]:
trainer.save_model("./hing-roberta") 

In [23]:
print(1)

1
